<a href="https://colab.research.google.com/github/JHyunjun/torch_GAN/blob/main/torch_GAN_celebA_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Created by Hyunjun JANG
#training GAN to check the performance with 3D-celeb image

import torch
import torch.nn as nn
import numpy 
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import random
import h5py
import zipfile
import torchvision.datasets
import os
import imageio
import time

# location of the HDF5 package, yours may be under /gan/ not /myo_gan/
hdf5_file = '/content/drive/MyDrive/Colab Notebooks/Data/img/celeba_aligned_small.h5py'

# how many of the 202,599 images to extract and package into HDF5
total_images = 20000

with h5py.File(hdf5_file, 'w') as hf:

    count = 0

    with zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/Data/img/source1.zip', 'r') as zf:
      for i in zf.namelist():
        if (i[-4:] == '.jpg'):
          # extract image
          ofile = zf.extract(i)
          img = imageio.imread(ofile)
          os.remove(ofile)

          # add image data to HDF5 file with new name
          hf.create_dataset('img_align_celeba/'+str(count)+'.jpg', data=img, compression="gzip", compression_opts=9)
          
          count = count + 1
          if (count%1000 == 0):
            print("images done .. ", count)
            pass
            
          # stop when total_images reached
          if (count == total_images):
            break
          pass

        pass
      pass

In [ ]:
# CUDA가 가능한지 확인
# 가능하다면, cuda에 기본 형태를 설정

if torch.cuda.is_available():
  torch.set_default_tensor_type(torch.cuda.FloatTensor)
  print("using cuda:", torch.cuda.get_device_name(0))
  pass

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

In [ ]:

with h5py.File(hdf5_file, 'r') as file_object:
  dataset = file_object['img_align_celeba']
  image = numpy.array(dataset['227.jpg'])
  print(image.shape)
  plt.imshow(image, interpolation='none')
  pass

In [ ]:
def crop_centre(img, new_width, new_height):
    height, width, _ = img.shape
    startx = width//2 - new_width//2
    starty = height//2 - new_height//2    
    return img[  starty:starty + new_height, startx:startx + new_width, :]

In [ ]:
class CelebADataset(Dataset):
    
    def __init__(self, file):
        self.file_object = h5py.File(file, 'r')
        self.dataset = self.file_object['img_align_celeba']
        pass
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, index):
        if (index >= len(self.dataset)):
          raise IndexError()
        img = numpy.array(self.dataset[str(index)+'.jpg'])
        # 128x128 사각형으로 크롭
        img = crop_centre(img, 128, 128)
        return torch.cuda.FloatTensor(img).permute(2,0,1).view(1,3,128,128) / 255.0
    
    def plot_image(self, index):
        img = numpy.array(self.dataset[str(index)+'.jpg'])
        # 128x128 사각형으로 크롭
        img = crop_centre(img, 128, 128)
        plt.imshow(img, interpolation='nearest')
        pass
    
    pass

In [ ]:
celeba_dataset = CelebADataset(hdf5_file)
celeba_dataset.plot_image(227)

In [ ]:
def generate_random(size) :
  random_data = torch.rand(size)
  return random_data

def generate_random_seed(size):
    random_data = torch.randn(size)
    return random_data  

def generate_random_image(size) : 
  random_data = torch.rand(size)
  return random_data

In [ ]:
class View(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape,

    def forward(self, x):
        return x.view(*self.shape)

In [ ]:
#Discriminator is learning about generate_real as true pattern and generate_random is false pattern

Data_size = 128*128*3
class Discriminator(nn.Module):
    
    def __init__(self):
        # initialise parent pytorch class
        super().__init__()
        
        # define neural network layers
        self.model = nn.Sequential(
            nn.Conv2d(3, 256, kernel_size=8, stride=2),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(256, 256, kernel_size=8, stride=2),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(256, 3, kernel_size=8, stride=2),
            nn.LeakyReLU(0.2),
            
            View(3*10*10),
            nn.Linear(3*10*10, 1),
            nn.Sigmoid()
        )
        
        # create loss function
        self.loss_function = nn.BCELoss()

        # create optimiser, simple stochastic gradient descent
        self.optimiser = torch.optim.Adam(self.parameters(), lr=0.0001)

        # counter and accumulator for progress
        self.counter = 0;
        self.progress = []

        pass
    
    
    def forward(self, inputs):
        # simply run model
        return self.model(inputs)
    
    
    def train(self, inputs, targets):
        # calculate the output of the network
        outputs = self.forward(inputs)
        
        # calculate loss
        loss = self.loss_function(outputs, targets)

        # increase counter and accumulate error every 10
        self.counter += 1;
        if (self.counter % 10 == 0):
            self.progress.append(loss.item())
            pass
        if (self.counter % 3000 == 0):
            print("counter = ", self.counter)
            pass

        # zero gradients, perform a backward pass, update weights
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()

        pass
    
    
    def plot_progress(self):
        df = pd.DataFrame(self.progress, columns=['Discriminator loss'])
        df.plot(ylim=(0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5, 1.0, 5.0))
        pass
    
    pass

In [ ]:
#Training the Discriminator
D = Discriminator()
D.to(device)
targets = torch.cuda.FloatTensor([1.0])
non_target = torch.cuda.FloatTensor([0.0])

for image_data_tensor in celeba_dataset : 
  D.train(image_data_tensor, targets)
  D.train(generate_random_image((1,3,128,128)), non_target)

  pass

In [ ]:
# Constructing Generator

class Generator(nn.Module) : 
  def __init__(self) : 
    super().__init__()

    self.model = nn.Sequential(
    
            nn.Linear(100, 3*11*11),
            nn.LeakyReLU(0.2),
            
         
            View((1, 3, 11, 11)),
            
            nn.ConvTranspose2d(3, 256, kernel_size=8, stride=2),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.ConvTranspose2d(256, 256, kernel_size=8, stride=2),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.ConvTranspose2d(256, 3, kernel_size=8, stride=2, padding=1),
            nn.BatchNorm2d(3), #1,3,128,128
        
            nn.Sigmoid() 
    )

    self.optimiser = torch.optim.Adam(self.parameters(), lr = 0.0001)
    self.counter = 0
    self.progress = []

    pass

  def forward(self, inputs) : 
    return self.model(inputs)

  def train(self, D, inputs, targets) : 
    g_output = self.forward(inputs)
    d_output = D.forward(g_output)
    loss = D.forward(g_output)
    loss = D.loss_function(d_output, targets)

    self.counter+=1;
    if (self.counter % 10 == 0) :
      self.progress.append(loss.item())
      pass

    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()

  def plot_progress(self):
        df = pd.DataFrame(self.progress, columns=['Generator loss'])
        df.plot(ylim=(0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5, 1.0, 5.0))
        
  pass

In [ ]:
#Check a Generator
G = Generator()
G.to(device)
output = G.forward(generate_random_seed(100))
img = output.detach().permute(0,2,3,1).view(128,128,3).cpu().numpy()

plt.imshow(img, interpolation='none', cmap='Reds')

In [ ]:
#Training both Generator and Discriminator for pattern
D = Discriminator()
D.to(device)
G = Generator()
G.to(device)

epochs = 1

for epoch in range(epochs) : 
  print("epoch = ", epoch+1)
  for image_data_tensor in celeba_dataset : 
    D.train(image_data_tensor, torch.cuda.FloatTensor([1.0]))
    D.train(G.forward(generate_random_seed(100)).detach(),torch.cuda.FloatTensor([0.0]))

    G.train(D, generate_random_seed(100), torch.cuda.FloatTensor([1.0]))
  
    pass
  pass

In [ ]:
#Plotting the Generator Loss
D.plot_progress()
G.plot_progress()

In [ ]:
#Checking the result of Generator

f, axarr = plt.subplots(2,3,figsize = (16,8))
for i in range(2) : 
  for j in range(3) : 
    output = G.forward(generate_random_seed(100))
    img = output.detach().permute(0,2,3,1).view(128,128,3).cpu().numpy()
    axarr[i,j].imshow(img, interpolation = 'none', cmap = 'Reds')
    pass

pass

In [ ]:
#Check1
seed1 = generate_random_seed(100)
out1 = G.forward(seed1)
img1 = out1.detach().permute(0,2,3,1).view(128,128,3).cpu().numpy()
plt.imshow(img1, interpolation='none', cmap='Blues')

In [ ]:
#Check2
seed2 = generate_random_seed(100)
out2 = G.forward(seed2)
img2 = out2.detach().permute(0,2,3,1).view(128,128,3).cpu().numpy()
plt.imshow(img2, interpolation='none', cmap='Blues')

In [ ]:
#Check3
count = 0

# plot a 3 column, 2 row array of generated images
f, axarr = plt.subplots(3,4, figsize=(16,8))
for i in range(3):
    for j in range(4):
        seed = seed1 + (seed2 - seed1)/11 * count
        output = G.forward(seed)
        img = output.detach().permute(0,2,3,1).view(128,128,3).cpu().numpy()
        axarr[i,j].imshow(img, interpolation='none', cmap='Blues')
        count = count + 1
        pass
    pass